In [1]:
import os

In [2]:
!pwd

/Users/dhyaneshanchula/Documents/Text-Summary-Generator/research


In [3]:
os.chdir("../")

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textSummarizer.constant import *
from textSummarizer.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
    # Create parent directory if it doesn't exist
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)

        if not os.path.exists(self.config.local_data_file):
            logger.info("Starting data download...")
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=str(self.config.local_data_file),
            )
            logger.info(
                f"Data downloaded successfully!\nFile: {filename}\nHeaders:\n{headers}"
            )
        else:
            logger.info("File already exists. Skipping download.")

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            logger.info("Extracting zip file...")
            zip_ref.extractall(unzip_path)
            logger.info(f"Extraction completed at location: {unzip_path}")


In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-18 23:13:50,122: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-01-18 23:13:50,123: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-18 23:13:50,125: INFO: common: Created directory at: artifacts]
[2026-01-18 23:13:50,126: INFO: 3348094866: Starting data download...]
[2026-01-18 23:13:53,134: INFO: 3348094866: Data downloaded successfully!
File: artifacts/data_ingestion/data.zip
Headers:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 925A:3BCFBA:33268A:7FCDD8:696D1B0A
Accept-Ranges: bytes
Date: Sun, 18 Jan 2026 17:43:51 GMT
Via: 1.1 varnish
X-Served-By: cache-del21732-DEL
X-Cache: HIT
X